<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/Fake_Data_Generator_for_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
import time
import json
import random as rand
import paho.mqtt.client as mqtt  # Import paho-mqtt

def generate_fake_sensor_data():
    """
    Generates fake sensor data for Temperature, Humidity, Distance, and DLIGHT.

    Returns:
        dict: A dictionary containing the sensor data.
    """
    temperature = round(rand.uniform(15, 35), 1)
    humidity = round(rand.uniform(30, 80), 1)
    distance = round(rand.uniform(10, 100), 1)
    dlight = round(rand.uniform(0, 1000), 1)  # Example DLIGHT value
    return {
        "Temperature": temperature,
        "Humidity": humidity,
        "Distance": distance,
        "DLIGHT": dlight,
    }

def create_mqtt_message(sensor_data, topic="braude/D106/indoor"):
    """
    Creates an MQTT message with a topic and JSON payload.

    Args:
        sensor_data (dict): The sensor data dictionary.
        topic (str, optional): The MQTT topic. Defaults to "braude/D106/indoor".

    Returns:
        tuple: A tuple containing the topic and the JSON payload string.
    """
    payload = json.dumps(sensor_data)  # Convert the dictionary to a JSON string
    return topic, payload

def on_connect(client, userdata, flags, rc):
    """Callback function for when the client connects to the broker."""
    if rc == 0:
        print("Connected to MQTT Broker (HiveMQ)!")
        client.subscribe("braude/D106/indoor")
        client.subscribe("braude/D106/outdoor")  # Subscribe to the outdoor topic
    else:
        print(f"Failed to connect, return code {rc}")

def on_disconnect(client, userdata, rc):
    """Callback function for when the client disconnects from the broker."""
    if rc != 0:
        print(f"Unexpected disconnection with code {rc}")

def on_message(client, userdata, msg):
    """Callback function for when a message is received."""
    print(f"Received message on topic: {msg.topic} with payload: {msg.payload.decode()}")



# Example of how to use the functions in a Jupyter cell:
if __name__ == '__main__':
    broker_address = "broker.hivemq.com"
    port = 1883
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_disconnect = on_disconnect
    client.on_message = on_message
    try:
        client.connect(broker_address, port)
    except Exception as e:
        print(f"Error connecting to broker: {e}")
    client.loop_start()

    try:
        while(True):
            sensor_data_indoor = generate_fake_sensor_data()
            sensor_data_outdoor = generate_fake_sensor_data() # Generate data

            indoor_topic, indoor_payload = create_mqtt_message(sensor_data_indoor, "braude/D106/indoor")
            outdoor_topic, outdoor_payload = create_mqtt_message({"DLIGHT": sensor_data_outdoor["DLIGHT"]}, "braude/D106/outdoor") #only DLIGHT


            client.publish(indoor_topic, indoor_payload)
            print(f"Published to {indoor_topic}: {indoor_payload}")
            client.publish(outdoor_topic, outdoor_payload)  # Publish to the outdoor topic
            print(f"Published to {outdoor_topic}: {outdoor_payload}")
            time.sleep(1)
    except KeyboardInterrupt:
        print("Interrupted")
    finally:
        client.disconnect()
        client.loop_stop()

<ipython-input-5-29a5b761c709>:62: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Published to braude/D106/indoor: {"Temperature": 32.9, "Humidity": 49.9, "Distance": 43.5, "DLIGHT": 417.6}
Published to braude/D106/outdoor: {"DLIGHT": 707.4}
Connected to MQTT Broker (HiveMQ)!
Published to braude/D106/indoor: {"Temperature": 23.2, "Humidity": 47.7, "Distance": 44.3, "DLIGHT": 653.4}
Published to braude/D106/outdoor: {"DLIGHT": 729.7}
Received message on topic: braude/D106/indoor with payload: {"Temperature": 23.2, "Humidity": 47.7, "Distance": 44.3, "DLIGHT": 653.4}
Received message on topic: braude/D106/outdoor with payload: {"DLIGHT": 729.7}
Published to braude/D106/indoor: {"Temperature": 15.7, "Humidity": 68.7, "Distance": 10.0, "DLIGHT": 202.5}
Published to braude/D106/outdoor: {"DLIGHT": 954.2}
Received message on topic: braude/D106/indoor with payload: {"Temperature": 15.7, "Humidity": 68.7, "Distance": 10.0, "DLIGHT": 202.5}
Received message on topic: braude/D106/outdoor with payload: {"DLIGHT": 954.2}
Published to braude/D106/indoor: {"Temperature": 18.6, "H